This notebook is used to determine which strategy is the best for any particular asset.

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/starting_portfolio.csv")

In [3]:
import yfinance as yf

start_date = "2024-03-01"
end_date = "2025-03-01" # Last day is exclusive

portfolio = {}
for asset, weight in list(map(list, df.values)):
    portfolio[asset] = {
        "data": yf.Ticker(asset).history(start=start_date, end=end_date, actions=False),
        "weight": weight,
        "strategy": None,
        "return": float("-inf"),
    }

In [4]:
# Commission fee based on Webull, which is known for low commission fees:
# https://www.webull.com.sg/pricing
# Regular and Extended Hours (04:00 - 20:00 EST)
# 0.025%*Total Trade Amount (Min. USD 0.50)

def commission(order_size, price):
    return max(0.5, abs(order_size) * price * 0.00025)

## Testing with one asset and one strategy

In [41]:
import pandas_ta as ta
from backtesting import Strategy

class Momentum2(Strategy):
    def init(self):
        self.ma = self.I(ta.sma, self.data.Close.s, length=20)
        self.std = self.I(ta.stdev, self.data.Close.s, length=20)
        self.threshold = 1.5

    def next(self):
        price = self.data.Close[-1]
        ma = self.ma[-1]
        std = self.std[-1]
        
        if not self.position:
            if price < ma - self.threshold * std:
                self.sell()
            elif price > ma + self.threshold * std:
                self.buy()
        else:
            if self.position.size > 0 and price >= ma:
                self.position.close()
            elif self.position.size < 0 and price <= ma:
                self.position.close()
            
            

from backtesting import Backtest

data = portfolio["AVAX-USD"]["data"]
data.index = data.index.values.astype("datetime64[D]")
bt = Backtest(
    data, Momentum2, cash=1000000, commission=commission
)
stats = bt.run()
bt.plot()

return_pct = stats["Return [%]"]
if return_pct >= portfolio["AVAX-USD"]["return"]:
    portfolio["AVAX-USD"]["strategy"] = bt._strategy.__name__
    portfolio["AVAX-USD"]["return"] = return_pct

In [42]:
stats

Start                     2024-03-01 00:00:00
End                       2025-02-28 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    31.78082
Equity Final [$]                1854531.37989
Equity Peak [$]                 1933392.02393
Commissions [$]                   42699.56891
Return [%]                           85.45314
Buy & Hold Return [%]               -60.77764
Return (Ann.) [%]                    85.45314
Volatility (Ann.) [%]                81.72812
CAGR [%]                             85.76808
Sharpe Ratio                          1.04558
Sortino Ratio                         3.87547
Calmar Ratio                          5.03205
Alpha [%]                            82.54615
Beta                                 -0.04783
Max. Drawdown [%]                   -16.98178
Avg. Drawdown [%]                    -4.40578
Max. Drawdown Duration       79 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                          

## Testing with all assets and one strategy

In [43]:
from backtesting import Backtest

from strategies.momentum import Momentum

strat = Momentum

for asset in portfolio:
    data = portfolio[asset]["data"]
    data.index = data.index.values.astype("datetime64[D]")
    bt = Backtest(
        data, strat, cash=1000000 * portfolio[asset]["weight"], commission=commission
    )
    stats = bt.run()
    # bt.plot()
    return_pct = stats["Return [%]"]
    if return_pct >= portfolio[asset]["return"]:
        portfolio[asset]["strategy"] = bt.strat.__name__
        portfolio[asset]["return"] = return_pct

In [44]:
all_assets_one_strategy = pd.DataFrame(
    [
        [
            asset,
            portfolio[asset]["weight"],
            portfolio[asset]["strategy"],
            portfolio[asset]["return"]
        ]
        for asset in portfolio
    ],
    columns=["asset", "weight", "strategy", "return"]
)

all_assets_one_strategy

,asset,weight,strategy,return
0,CZR,0.195536,Scalping,4.694043
1,INTC,0.159487,RSIDivergence,17.569500
2,MHK,0.115818,Scalping,8.844675
3,BLDR,0.107738,RSIDivergence,25.506435
4,URI,0.088408,MichaelHarrisPriceAction,14.858892
5,ON,0.070599,RSIDivergence,38.940199
6,NCLH,0.031879,LarryWilliamsPriceAction,58.419585
7,ALB,0.028309,RSIDivergence,41.886634
8,VST,0.005340,LarryWilliamsPriceAction,122.958732
9,AVAX-USD,0.118918,Momentum2,85.453138


## All assets and all strategies

In [45]:
from backtesting import Backtest

from strategies.larry_williams_price_action import LarryWilliamsPriceAction
from strategies.macd_bollinger_bands_mean_reversion import MACDBollingerBandsMeanReversion
from strategies.michael_harris_price_action import MichaelHarrisPriceAction
from strategies.rsi_divergence import RSIDivergence
from strategies.scalping import Scalping
from strategies.volume_spike_reversal import VolumeSpikeReversal
from strategies.bollinger_bands_breakout import BollingerBandsBreakout

strategies = [
    BollingerBandsBreakout,
    LarryWilliamsPriceAction,
    MACDBollingerBandsMeanReversion,
    MichaelHarrisPriceAction,
    RSIDivergence,
    Scalping,
    VolumeSpikeReversal,
]

for asset in portfolio:
    for strategy in strategies:
        data = portfolio[asset]["data"]
        data.index = data.index.values.astype("datetime64[D]")
        bt = Backtest(
            data, strategy, cash=1000000 * portfolio[asset]["weight"], commission=commission
        )
        stats = bt.run()
        return_pct = stats["Return [%]"]
        if return_pct >= portfolio[asset]["return"]:
            portfolio[asset]["strategy"] = bt._strategy.__name__
            portfolio[asset]["return"] = return_pct

# NOTE: Many instances of broker cancelling the relative-sized order due to insufficient margin.

Backtest.run:   0%|          | 0/249 [00:00<?, ?it/s]c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:955: UserWarning: time=29: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:955: UserWarning: time=30: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:955: UserWarning: time=31: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:955: UserWarning: time=32: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\

In [46]:
results = pd.DataFrame(
    [
        [
            asset,
            portfolio[asset]["weight"],
            portfolio[asset]["strategy"],
            portfolio[asset]["return"]
        ]
        for asset in portfolio
    ],
    columns=["asset", "weight", "strategy", "return"]
)

results.to_csv("../data/processed/asset_strategies.csv", index=False)